In [1]:
import math, datetime, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


def load_data(data_path):
  data = pd.read_csv(data_path)  
  return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]

## Plan de campagne:

- Process the weather data into workable data for the algo
- Calculate on a minute-to-minute basis how many products are 'new' in the warehouse.
- Try to process the inbound and outbound data to make the amounts of incoming and outgoing products available
- Try to estimate on a minute-to-minute basis how many doors are open at any point in time.
    - This should be done on a percentage-based scale (how much percent of the minute was a door open)

In [2]:
# for data visualization
import math, datetime, time, random
import matplotlib.pyplot as plt
import missingno

In [4]:
def addtimecol(df, colname): ####input df and colname 
    df[colname] = pd.to_datetime(df[colname])         
    df['year'] = df[colname].dt.year
    df['month'] = df[colname].dt.month
    df['weekday'] = df[colname].dt.weekday
    df['day'] = df[colname].dt.day
    df['hour'] = df[colname].dt.hour
    df['minute'] = df[colname].dt.minute        
    return df

#Create new dummy dfs
base_df = train.copy()
base_weather = weather.copy()

#Remove unnecessary columns
base_df = base_df.drop('Unnamed: 0', axis=1)
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])
base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})
#base_weather['datetime_local'] = [datetime.datetime.strptime(x, 'yyyy/MM/dd HH:mm:SS') for x in base_weather['localstrptime']]

#Add time columns.
addtimecol(base_df, 'datetime_local')

#Set index to datetime
base_df.set_index('datetime_local', inplace=True)
base_weather.set_index('datetime_local', inplace=True)

#Concatenate the weather DataFrame to the base DataFrame
base_df = pd.concat([base_df, base_weather], axis=1)

# Drop all NaN values\n"
base_df.dropna(subset=['demand_kW', 'Temperature'])

,demand_kW,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,hour
datetime_local,,,,,,,,,,,,
2018-12-31 21:15:00,2064.101392,2018.0,12.0,0.0,31.0,21.0,15.0,2018-12-31 21:15:00-06:00,61.27,46.40,2019-01-01 03:15:00,21.0
2018-12-31 21:30:00,1874.002081,2018.0,12.0,0.0,31.0,21.0,30.0,2018-12-31 21:30:00-06:00,61.27,46.40,2019-01-01 03:30:00,21.0
2018-12-31 21:45:00,1988.168511,2018.0,12.0,0.0,31.0,21.0,45.0,2018-12-31 21:45:00-06:00,65.60,44.60,2019-01-01 03:45:00,21.0
2018-12-31 22:00:00,2022.795943,2018.0,12.0,0.0,31.0,22.0,0.0,2018-12-31 22:00:00-06:00,65.60,44.60,2019-01-01 04:00:00,22.0
2018-12-31 22:15:00,1986.981872,2018.0,12.0,0.0,31.0,22.0,15.0,2018-12-31 22:15:00-06:00,65.60,44.60,2019-01-01 04:15:00,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-11 05:50:00,2527.739000,2021.0,10.0,0.0,11.0,5.0,50.0,2021-10-11 06:50:00-05:00,87.75,57.20,2021-10-11 11:50:00,6.0
2021-10-11 05:53:00,2359.505000,2021.0,10.0,0.0,11.0,5.0,53.0,2021-10-11 06:53:00-05:00,83.84,57.92,2021-10-11 11:53:00,6.0
2021-10-11 05:55:00,2156.104000,2021.0,10.0,0.0,11.0,5.0,55.0,2021-10-11 06:55:00-05:00,87.75,57.20,2021-10-11 11:55:00,6.0


In [5]:
dummy_df = base_df.dropna(subset=['demand_kW', 'Temperature', 'Relative Humidity'])
dummy_df = dummy_df.reset_index()
dummy_df = dummy_df.drop(['hour'], axis=1)
dummy_df = dummy_df.drop(['datetime'], axis=1)
dummy_df = dummy_df.drop(['datetime_local'], axis=1)
dummy_df = dummy_df.drop(['datetime_UTC'], axis=1)
dummy_df

,demand_kW,year,month,weekday,day,minute,Relative Humidity,Temperature
0,2064.101392,2018.0,12.0,0.0,31.0,15.0,61.27,46.40
1,1874.002081,2018.0,12.0,0.0,31.0,30.0,61.27,46.40
2,1988.168511,2018.0,12.0,0.0,31.0,45.0,65.60,44.60
3,2022.795943,2018.0,12.0,0.0,31.0,0.0,65.60,44.60
4,1986.981872,2018.0,12.0,0.0,31.0,15.0,65.60,44.60
...,...,...,...,...,...,...,...,...
116228,2527.739000,2021.0,10.0,0.0,11.0,50.0,87.75,57.20
116229,2359.505000,2021.0,10.0,0.0,11.0,53.0,83.84,57.92
116230,2156.104000,2021.0,10.0,0.0,11.0,55.0,87.75,57.20
116231,2259.711000,2021.0,10.0,0.0,11.0,0.0,87.75,57.20


# Model Building

First we build the skeleton:
- Divide into train/test
- Set target column
- Get Accuracy

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error

#Define train, test sets\n",
train, test = train_test_split(dummy_df)
X_train = train.copy().drop(['demand_kW'], axis=1)
Y_train = train['demand_kW']
X_test = test.copy().drop(['demand_kW'], axis=1)
Y_test = test['demand_kW']

In [ ]:
svm = SVR()
svm.fit(X_train, Y_train)

In [ ]:
acc = svm.predict(X_test)
result = r2_score(list(Y_test), acc)
result